In [52]:
## The usual imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
## for printing image
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np

In [53]:
print(torch.__version__)

1.1.0.post2


In [54]:
device = torch.device("cuda:0" if torch.cuda.is_available() else"cpu") 

In [55]:
torch.cuda.is_available() 

False

In [56]:
## parameter denoting the batch size 
BATCH_SIZE = 32 
## transformations Compose 将多个transform 放在参数的list中
transform = transforms.Compose([transforms.ToTensor()]) 
## download and load training dataset 
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform) 
# data loader 本身只是一个lodaer 记录了一些 数据的batch_size 以及是否shuffle等信息 以及关联的数据
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2) 
## download and load testing dataset 
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform) 
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [57]:
type(trainset)
type(trainloader)

torch.utils.data.dataloader.DataLoader

In [58]:
# train loader 实际上是一个包装好的迭代器
# torch.Size([32, 1, 28, 28])
for images, labels in trainloader: 
    print("Image batch dimensions:", images.shape) 
    print("Image label dimensions:", labels.shape) 
    print(labels)
    break

Image batch dimensions: torch.Size([32, 1, 28, 28])
Image label dimensions: torch.Size([32])
tensor([9, 9, 5, 5, 9, 8, 6, 8, 6, 4, 7, 1, 2, 7, 0, 7, 9, 5, 9, 4, 4, 1, 8, 2,
        9, 0, 6, 3, 6, 4, 7, 7])


labels.dtypes

In [59]:
#label_dict = {2: 0, 3: 1,  4:2,  5:3,  6:4,  8:5,  9:6, 10:7, 12:8, 13:9, 16:10, 17:11, 18:12, 25:13, 31:14, 38:15, 39:16, 42:17, 22:18}
jpg_path = "./data/GTSRB_train_jpgs/"
files = [file for file in os.listdir(jpg_path) if ".jpg" in file]
std_size = (28, 28)
images = [cv2.cvtColor(cv2.resize(cv2.imread(jpg_path + file), std_size), cv2.COLOR_BGR2GRAY).reshape(1, std_size[1], std_size[0]) for file in files]
labels = [file.split("#")[0] for file in files]

In [ ]:
def tain_data_loader(train_set):
    

#### 继承nn.Module 模块定义一个神经网络的前馈结构

In [29]:
## the model 
class MyModel(nn.Module): 
    def __init__(self): 
        # 在init中定义 各层对象结构
        super(MyModel, self).__init__() 
        # 就是一般的全连接层 第一个参数是输入维度 第二个参数是输出维度
        # 注意卷积层的输入为尺寸为 (N, Channel, Height, Width)
        # 
        self.c1 = torch.nn.Conv2d(1, 6, (5, 5), stride=1, padding=0, bias=True, padding_mode='zeros')
        self.s2 = torch.nn.MaxPool2d((2, 2))
        self.c3 = torch.nn.Conv2d(6, 16, (5, 5), stride=1, padding=0, bias=True, padding_mode='zeros')
        self.s4 = torch.nn.MaxPool2d((2, 2))
        self.d1 = nn.Linear(256, 120) 
        #self.dropout = nn.Dropout(p=0.2) 
        self.d2 = nn.Linear(120, 84) 
        self.d3 = nn.Linear(84, 10)
        
    def forward(self, x): 
        # 使用init中创建的层对象 构建网络的连接关系
        #print("input_shape", x.shape)
        x =  F.relu(self.s2(self.c1(x)))
        x =  F.relu(self.s4(self.c3(x)))
        x = x.view(-1, 256) # 把3d(depth,x,y)的图像数据展平为一维的 这里用的就是普通神经网络不是CNN
        x = F.relu(self.d1(x))  # 全连接层1
        x = F.relu(self.d2(x))  # 全连接层1
        x = self.d3(x)
        out = F.log_softmax(x, dim=1)
        #exit()
        return out
        #return out

#### 开始训练模型

In [30]:
learning_rate = 0.001 
num_epochs = 5 
device = torch.device("cuda:0" if torch.cuda.is_available() else"cpu") 
model = MyModel() 
model = model.to(device) # 模型放到CPU 或者GPU上
criterion = nn.CrossEntropyLoss() # 定义损失函数
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [31]:
## utility function to compute accuracy 
# 自己写一个计算准确度的函数
def get_accuracy(output, target, batch_size):
    ''' Obtain accuracy for training round '''
    corrects = (torch.max(output, 1)[1].view(target.size()).data == target.data).sum() 
    accuracy = 100.0 * corrects/batch_size 
    return accuracy.item()

In [32]:
## train the model 
for epoch in range(num_epochs): 
    train_running_loss = 0.0 
    train_acc = 0.0 
     ## commence training 
    model = model.train() 
     ## training step 
    for i, (images, labels) in enumerate(trainloader): 
        images = images.to(device) 
        labels = labels.to(device) 
         ## forward + backprop + loss 
        predictions = model(images) 
        #print(predictions, predictions.shape)
        #print(labels, labels.shape)
        #break
        loss = criterion(predictions, labels) 
        optimizer.zero_grad()
        loss.backward()
        ## update model params 
        optimizer.step()
        train_running_loss += loss.detach().item() 
        train_acc += get_accuracy(predictions, labels, BATCH_SIZE) 
    model.eval()
    print('Epoch: %d | Loss: %.4f | Train Accuracy: %.2f' %(epoch,train_running_loss / i, train_acc/i))

Epoch: 0 | Loss: 0.2493 | Train Accuracy: 91.62
Epoch: 1 | Loss: 0.0774 | Train Accuracy: 97.23
Epoch: 2 | Loss: 0.0552 | Train Accuracy: 97.91
Epoch: 3 | Loss: 0.0431 | Train Accuracy: 98.42
Epoch: 4 | Loss: 0.0368 | Train Accuracy: 98.58


In [33]:
test_acc = 0.0 
for i, (images, labels) in enumerate(testloader, 0): 
    images = images.to(device) 
    labels = labels.to(device) 
    outputs = model(images) 
    test_acc+= get_accuracy(outputs, labels, BATCH_SIZE) 
print('TestAccuracy: %.2f' % (test_acc / i))

TestAccuracy: 98.91
